In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
%pylab inline
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from numpy.linalg import norm
from numpy import copy

from scipy.stats import probplot
import scipy as sp

from ipywidgets import FloatProgress
from IPython.display import display

import timeit

import fibonacci_heap_mod as fhm

from scipy.sparse import csr_matrix
from scipy import sparse

import sys

Populating the interactive namespace from numpy and matplotlib


## Генерация задачи

In [220]:
%%time

n = 1e5
m = n
mu = 1/n
X = sparse.rand(m, n, density=0.001)
#x_true = np.random.rand(int(n))
#x_true /= x_true.sum()
#y = X.dot(x_true.T)
#e_lower_lim = X.max()
print(len(X.nonzero()[0]))
#x0 = np.random.rand(int(n))
#x0 /= sum(x0)

10000000
CPU times: user 39.3 s, sys: 2.44 s, total: 41.8 s
Wall time: 44.6 s


## Арора-спарсификация

In [56]:
import numpy as np
from scipy import sparse

class SparsificationException(Exception):
    pass


def arora_sparsification(M, e = None, random_seed = None):
    if random_seed is not None:
        np.random.seed(random_seed)

    if e is None:
        e = M.max()

    A = sparse.lil_matrix(M.copy())

    for i, j in zip(*A.nonzero()):
        x = A[i, j]
        if abs(x) > e:
            continue
        p = abs(x)/e
        if p > 1 or p < 0:
            raise SparsificationException("Inadequate probability on (%d, %d): %.2f"%(i, j, p))
        if np.random.rand() <= p:
            A[i, j] = np.sign(x)*e
        else:
            A[i, j] = 0

    return A

### Тест быстродействия

In [69]:
%%time

X_b = arora_sparsification(X, e_lower_lim*5)

CPU times: user 39.1 s, sys: 998 ms, total: 40.1 s
Wall time: 41.1 s


In [70]:
print("%d of non-zero elements before"%(len(X.nonzero()[0])))

2000000 of non-zero elements before


In [71]:
print("%d of non-zero elements after"%(len(X_b.nonzero()[0])))

199576 of non-zero elements after


## Бернштейн-спарсификация

In [141]:
from numpy.random import choice

In [142]:
%%time
X_1st_norm = X.sum()

CPU times: user 647 µs, sys: 146 µs, total: 793 µs
Wall time: 525 µs


In [143]:
%%time
X = X.tolil()
probabilities = [X[i,j]/X_1st_norm for (i, j) in zip(*X.nonzero())]
X = X.tocsr()

CPU times: user 709 ms, sys: 12 ms, total: 721 ms
Wall time: 727 ms


In [197]:
e = 0.001
n_samples = int(np.log(n+m)*max(n,m)/e**2)
n_samples

7600902459

In [186]:
np.linalg.norm((X/X_1st_norm).todense(), ord=2)

0.0010119005217629064

In [187]:
%%time 
sparsified_indices = choice(arange(0, len(X.nonzero()[1])), n_samples, probabilities)

CPU times: user 3.85 ms, sys: 2.63 ms, total: 6.48 ms
Wall time: 4.65 ms


In [189]:
%%time
indices = list(zip(*X.nonzero()))
X_bern = sparse.lil_matrix(X.shape)
for k in sparsified_indices:
    i, j = indices[k]
    X_bern[i, j] += 1
    
X_bern = X_bern.tocsr()

X_bern /= n_samples

CPU times: user 130 ms, sys: 8.38 ms, total: 139 ms
Wall time: 139 ms


In [190]:
X_bern

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 7296 stored elements in Compressed Sparse Row format>

In [191]:
e_real = np.linalg.norm((X/X_1st_norm - X_bern).todense(), ord=2)
print("%.5f < %.5f"%(e_real, sqrt(2)*e + e**2/3))

0.00080 < 1.74755


Неравенство Бернштейна выполняется, но является слишком грубой оценкой

## Метрополис-Хастингс

In [216]:
int(n*log(n))

1151292

In [221]:
%%time

X = X.tolil()

if X.shape[0] != X.shape[1]:
    print("m != n")
    
k = 0
b_prev = None
X_mh = sparse.lil_matrix(X.shape)
indices = list(zip(*X.nonzero()))
already_got = np.zeros(len(indices))
while k <= int(n*log(n)):
    pt = np.random.randint(low = 0, high = len(indices))
    if already_got[pt]:
        continue
    i, j = indices[pt]
    b = X[i, j]
    if b_prev == None:
        X_mh[i, j] = sign(b)
    else:
        if random.rand() < min(abs(b)/abs(b_prev), 1):
            X_mh[i, j] = sign(b)
        else:
            continue
    b_prev = b
    already_got[pt] = 1
    k += 1

X = X.tocsr()

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
CPU times: user 59.9 s, sys: 4.81 s, total: 1min 4s
Wall time: 1min 5s
